In [86]:
#imports
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal, stats #filters
import scipy.fftpack #fft
from scipy.signal import chirp, find_peaks, peak_widths, peak_prominences

np.random.seed(1)

In [92]:
#find files
#function to define path
folder = "data_text_files/"
rats = os.listdir(folder)
rats = list(filter(lambda x: x!=".DS_Store", rats))
print(rats)

def findfiles(rat, freq):
    path = folder + rat + "/"
    file = path + str(freq) +'.txt'
    return(file)

def skiplines(row_text):
    line = 0
    while True:
        if fh.readline().startswith(row_text):
            break
        line += 1
    return(line)

#load files into pandas data frame
df_all = pd.DataFrame({'ratname':[], 'date':[], 'frequency':[],'dB':[],'L/R':[],'ABR':[], 'filtered':[], 'label':[]})

freqs = [0, 2000, 4000, 8000, 16000, 24000, 32000]
# dbs = [90, 85, 80, 75, 70, 65, 60, 55, 50, 45, 40, 35, 30, 25, 20, 15] --> read from file some stop at 25, 20 or even 15 sometimes

f = 0
i = 0
for rat in rats:
    for f in freqs:
        #find file
        file = findfiles(rat, i)

        #open file
        fh = open(file, "r")

        #find abr data
        line = skiplines('Data')
        data = pd.io.parsers.read_csv(file, skiprows=line)
        #Keeping only the column containing the average 
        data = data[[c for c in data.columns if c.startswith('Average(uV):')]]
        data.columns = np.arange(len(data.columns))

        #check line index
        fh = open(file, "r")
        line = skiplines('Intensity:')
        # print(line)
        db = (pd.io.parsers.read_csv(file, skiprows = line-1, header=None))[1:2].dropna(axis = 1)
        dbs = np.array(db)[:,1:].astype('int')

        print(dbs)
        #load entries
 




['R165183_042022', 'R144883_091321', 'R165184_042222', 'R144884_091321', 'R165182_042022', 'R165181_041322', 'R165176_04102022', 'R144863_090921', 'R165185_042222']
[[90 85 80 75 70 65 60 55 50 45 40 35 30 25 20 90 85 80 75 70 65 60 55 50
  45 40 35 30 25 20]]
[[90 85 80 75 70 65 60 55 50 45 40 35 30 25 20 90 85 80 75 70 65 60 55 50
  45 40 35 30 25 20]]
[[90 85 80 75 70 65 60 55 50 45 40 35 30 25 20 90 85 80 75 70 65 60 55 50
  45 40 35 30 25 20]]
[[90 85 80 75 70 65 60 55 50 45 40 35 30 25 20 90 85 80 75 70 65 60 55 50
  45 40 35 30 25 20]]
[[90 85 80 75 70 65 60 55 50 45 40 35 30 25 20 90 85 80 75 70 65 60 55 50
  45 40 35 30 25 20]]
[[90 85 80 75 70 65 60 55 50 45 40 35 30 25 20 90 85 80 75 70 65 60 55 50
  45 40 35 30 25 20]]
[[90 85 80 75 70 65 60 55 50 45 40 35 30 25 20 90 85 80 75 70 65 60 55 50
  45 40 35 30 25 20]]
[[90 85 80 75 70 65 60 55 50 45 40 35 30 25 20 90 85 80 75 70 65 60 55 50
  45 40 35 30 25 20]]
[[90 85 80 75 70 65 60 55 50 45 40 35 30 25 20 90 85 80 75 70 65 60

In [ ]:
df_all

,ratname,date,frequency,dB,L/R,ABR,filtered,label


In [ ]:
#signal processing 

#filter data

In [ ]:
#calculate noise floor
#determine threshold

# ONLY signal is present data as input for find peaks 

In [ ]:
#find peaks

#select peaks of interest

In [ ]:
#peak prominencces to detect node to peak (amplitude)
#find node latency